### 边预测任务实践
边预测任务，目标是预测两个节点之间是否存在边。拿到一个图数据集，
我们有节点属性x，边端点edge_index 。edge_index 存储的便是正样本。为了构建边预测任务，我们需要生成一些负样本，即采样一些不存在
边的节点对作为负样本边，正负样本数量应平衡。此外要将样本分为训练集、验证集和测试集三个集合

PyG中为我们提供了现成的采样负样本边的方法，
train_test_split_edges(data, val_ratio=0.05, test_ratio=0.1)，其

第一个参数为torch_geometric.data.Data对象，
第二参数为验证集所占比例，
第三个参数为测试集所占比例。

该函数将自动地采样得到负样本，并将正负样本分成训练集、验证集和测

试集三个集合。它用train_pos_edge_index 、train_neg_adj_mask、 val_pos_edge_index、val_neg_edge_index、test_pos_edge_index 和test_neg_edge_index ，六个属性取代edge_index 属性。
注意train_neg_adj_mask与其他属性格式不同，其实该属性在后面并没
有派上用场，后面我们仍然需要进行一次训练集负样本采样

### 获取数据集并实践
使用Cora数据集作为例子，进行边预测任务说明

In [1]:
import os.path as osp
from torch_geometric.utils import negative_sampling
from torch_geometric.datasets import Planetoid
import torch_geometric.transforms as T
from torch_geometric.utils import train_test_split_edges

# 加载数据集
dataset = Planetoid(root='K:/CodeWorkSpace/DeeplApp/graph-network-code/datasets',
                    name='Cora', transform=T.NormalizeFeatures())
data = dataset[0]
# 不在使用节点属性，只使用边属性
data.train_mask = data.val_mask = data.test_mask = None
# 打印边的形状
print(data.edge_index.shape)

torch.Size([2, 10556])


In [2]:
# 将边分为训练集、验证集和测试集
data = train_test_split_edges(data)

# 打印所有边的信息
for key in data.keys:
    print(key, getattr(data, key).shape)

val_neg_edge_index torch.Size([2, 263])
y torch.Size([2708])
val_pos_edge_index torch.Size([2, 263])
x torch.Size([2708, 1433])
test_neg_edge_index torch.Size([2, 527])
train_neg_adj_mask torch.Size([2708, 2708])
train_pos_edge_index torch.Size([2, 8976])
test_pos_edge_index torch.Size([2, 527])


D:\DevApp\anaconda3\envs\Graph_Network\lib\site-packages\torch_geometric\deprecation.py:13: UserWarning: 'train_test_split_edges' is deprecated, use 'transforms.RandomLinkSplit' instead
  warnings.warn(out)


我们观察到训练集、验证集和测试集中正样本边的数量之和不等于原始边
的数量。这是因为，现在所用的Cora图是无向图，在统计原始边数量时，
每一条边的正向与反向各统计了一次，训练集也包含边的正向与反向，但
验证集与测试集都只包含了边的一个方向

为什么训练集要包含边的正向与反向，而验证集与测试集都只包含了边的
一个方向？

这是因为，训练集用于训练，训练时一条边的两个端点要互传信息，只考虑一个方向的话，只能由一个端点传信息给另一个端点
而验证集与测试集的边用于衡量检验边预测的准确性，只需考虑一个方向的边即可。

### 构造边预测神经网络

节点表征 : 节点的特征表示
参考 : https://zhuanlan.zhihu.com/p/306261981

Graph的特征表示极为复杂，主要表现在以下三个方面：

极其复杂的拓扑结构，很难简单地像图像中的感受野来提取有效信息；
无特定的节点顺序；
通常graph会是动态变化的， 且使用多模态特征；

In [3]:
import torch
import torch.nn as nn
from torch_geometric.nn import GCNConv


class EdgeNet(nn.Module):

    def __init__(self, in_channels, out_channels):
        """
        :param in_channels:
        :param out_channels:
        """
        super(EdgeNet, self).__init__()
        self.conv1 = GCNConv(in_channels=in_channels, out_channels=128)
        self.conv2 = GCNConv(in_channels=128, out_channels=out_channels)

    def encode(self, x, edge_index):
        """
        生成节点表征(节点的特征表示)
        :param x: 节点矩阵 [2708, 1433]
        :param edge_index: 边下标 [2, ]
        :return:
        """
        x = self.conv1(x, edge_index)
        x = x.relu()
        x = self.conv2(x, edge_index)
        return x

    def decode(self, z, pos_edge_index, neg_edge_index):
        """
        根据边两端节点的表征生成边为真的几率（odds）
        :param z:
        :param pos_edge_index: 例如 : [2, 527]
        :param neg_edge_index: 例如 : [2, 527]
        :return:
        """
        # 拼接正向边和负向边
        edge_index = torch.cat([pos_edge_index, neg_edge_index], dim=-1) # [2, 1054]
        return z[edge_index[0]] * z[edge_index[1]].sum(dim=-1) #

    def decode_all(self, z):
        """
        推理阶段 : 对所有的节点预测存在边的几率
        :param z:
        :return:
        """
        # 矩阵相乘
        prob_obj = z @ z.t()
        return (prob_obj > 0).nozero(as_tuple=False).t()


### 边预测图神经网络的训练


In [4]:
import torch.functional as F

# 定义单个epoch训练过程
def get_link_label(pos_edge_index, neg_edge_index):
    """
    函数用于生成完整训练集的标签。
    :param pos_edge_index:
    :param neg_edge_index:
    :return:
    """
    # 统计正向边和负向边的数量
    num_links = pos_edge_index.size(1) + neg_edge_index.size(1)
    # 获取节点标签
    link_labels = torch.zeros(num_links, dtype=torch.float)
    link_labels[:pos_edge_index.size(1)] = 1
    return link_labels

def train(data, model, optimizer):
    model.train()
    # 负向采样
    neg_edge_index = negative_sampling(edge_index=data.train_pos_edge_index,
                                       num_nodes=data.num_nodes,
                                       num_neg_samples=data.train_pos_edge_index)
    # 梯度归零
    optimizer.zero_grad()
    # 编码器, 生成节点表征
    z = model.encode(data.x, data.train_pos_edge_index)

    # TODO : ??????

    # 解码
    link_logits = model.decode(z, data.train_pos_edge_index, neg_edge_index)
    #
    link_labels = get_link_label(data.train_pos_edge_index, neg_edge_index).to(data.x.device)
    # 计算目标和输入之间的二进制交叉熵
    loss = F.binary_cross_entropy_with_logits(link_logits, link_labels)
    loss.backward()
    optimizer.step()
    return loss


通常，存在边的节点对的数量往往少于不存在边的节点对的数量。

我们在每一个epoch的训练过程中，都进行一次训练集负样本采样。

采样到的样本数量与训练集正样本相同，但不同epoch中采样到的样本是不同的。

这样做，我们既能实现类别数量平衡，又能实现增加训练集负样本的多样性。

在负样本采样时，我们传递了train_pos_edge_index 为参数，
于是negative_sampling() 函数只会在训练集中不存在边的节点对中采样。

get_link_labels() 函数用于生成完整训练集的标签。
注：在训练阶段，我们应该只见训练集，对验证集与测试集都是不可见的。

所以我们没有使用所有的边，而是只用了训练集正样本边。定义单个epoch验证与测试过程